In [1]:
import pandas as pd
import pickle
import zlib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Create district map
df = pd.read_csv('Files/pace.csv')
dist_map = {}
census = list(df['census_code'])
label = list(df['labels'])
for i in range(len(census)):
    dist_map[census[i]] = label[i] 

In [4]:
#Update dataset using new ADI values
import pandas as pd
datasets = ['dataset_agriculture', 'dataset_development', 'dataset_environment', 'dataset_industrialization', 'dataset_lifestyle']
model_names = ['model_agriculture', 'model_development', 'model_environment', 'model_industrialization', 'model_lifestyle']
final_res_all = []
for dataset, model in zip(datasets,model_names):
    dataset_name = dataset
    # Printing the collection name.
    collection_name = dataset[8:]
    print('\nCollection:',collection_name.capitalize())

    # Loading the dataset and the model from the drive.
    file = open('../Datasets/'+dataset, 'rb')
    dataset = pickle.loads(zlib.decompress(pickle.load(file)))
    file.close()
    #model = Doc2Vec.load('./Models/'+model)
    new_dataset = []
    for i in dataset:
        if i[-4] in dist_map:
            i.append(dist_map[i[-4]])
            new_dataset.append(i)
    
    print(len(new_dataset))
    file = open('Datasets/dataset_'+collection_name,'wb')
    pickle.dump(zlib.compress(pickle.dumps(new_dataset),pickle.HIGHEST_PROTOCOL),file,pickle.HIGHEST_PROTOCOL)
    file.close()
    
    


Collection: Agriculture
128487

Collection: Development
25709

Collection: Environment
155205

Collection: Industrialization
172430

Collection: Lifestyle
370509


In [5]:
#Print count of articles for each subclass
datasets = ['dataset_agriculture', 'dataset_development', 'dataset_environment', 'dataset_industrialization', 'dataset_lifestyle']
all_cnt = []
for dataset in datasets:
    file = open('Datasets/'+dataset,'rb')
    dataset = pickle.loads(zlib.decompress(pickle.load(file)))
    file.close()
    counts = {}

    for i in dataset:
        if i[6]+' '+i[-1] in counts:
            counts[i[6]+' '+i[-1]]+=1
        else:
            counts[i[6]+' '+i[-1]]=1
    print(counts,'\n\n')
    all_cnt.append(counts)

{'Non Agri Fast': 69581, 'Unemp Slow': 11097, 'Agri Average': 12204, 'Unemp Average': 5968, 'Agri Slow': 9639, 'Non Agri Slow': 6762, 'Unemp Fast': 3968, 'Agri Fast': 4183, 'Non Agri Average': 5085} 


{'Non Agri Fast': 15477, 'Unemp Slow': 2160, 'Unemp Fast': 647, 'Non Agri Slow': 1633, 'Non Agri Average': 1199, 'Unemp Average': 1085, 'Agri Average': 1565, 'Agri Slow': 1373, 'Agri Fast': 570} 


{'Unemp Slow': 10248, 'Non Agri Fast': 97692, 'Unemp Fast': 5020, 'Non Agri Slow': 9776, 'Agri Average': 8665, 'Agri Fast': 3518, 'Non Agri Average': 5890, 'Agri Slow': 7734, 'Unemp Average': 6662} 


{'Non Agri Fast': 128383, 'Unemp Fast': 3648, 'Agri Fast': 2084, 'Agri Average': 6269, 'Non Agri Slow': 8337, 'Non Agri Average': 4964, 'Unemp Slow': 8379, 'Agri Slow': 5798, 'Unemp Average': 4568} 


{'Unemp Slow': 29770, 'Non Agri Fast': 244454, 'Non Agri Slow': 26743, 'Agri Slow': 14513, 'Unemp Fast': 10458, 'Agri Average': 12628, 'Non Agri Average': 12748, 'Unemp Average': 13957, 'Agri Fast':

In [6]:
keys = sorted(all_cnt[0].keys())
n = ['Agri Fast',
 'Agri Average',
 'Agri Slow',
 'Non Agri Fast',
 'Non Agri Average',
 'Non Agri Slow',
 'Unemp Fast',
 'Unemp Average',
 'Unemp Slow']
#print([n[-1]],' & ',[n[-2]],' & ',[n[-3]],' & ',[n[2]],' & ',[n[1]],' & ',[n[0]],' & ',[n[-4]],' & ',[n[-5]],' & ',[n[-6]])
for i in all_cnt:
    print(i[n[-1]],' & ',i[n[-2]],' & ',i[n[-3]],' & ',i[n[2]],' & ',i[n[1]],' & ',i[n[0]],' & ',i[n[-4]],' & ',i[n[-5]],' & ',i[n[-6]])

11097  &  5968  &  3968  &  9639  &  12204  &  4183  &  6762  &  5085  &  69581
2160  &  1085  &  647  &  1373  &  1565  &  570  &  1633  &  1199  &  15477
10248  &  6662  &  5020  &  7734  &  8665  &  3518  &  9776  &  5890  &  97692
8379  &  4568  &  3648  &  5798  &  6269  &  2084  &  8337  &  4964  &  128383
29770  &  13957  &  10458  &  14513  &  12628  &  5238  &  26743  &  12748  &  244454


In [6]:
def DT2V_train(collection_name):

    print(collection_name.capitalize())

    # Loading data from drive.
    file = open('Datasets/dataset_' + collection_name,'rb')
    dataset = pickle.loads(zlib.decompress(pickle.load(file)))
    file.close()

    # Creating the documents.
    documents = [TaggedDocument(i[3],[i[0],i[5],i[6],i[7],i[-1]]) for i in dataset]
    print('Documents Collected.')

    # Declaring the DT2V Model.
    model = Doc2Vec(vector_size=50,window=3,min_count=3,alpha=0.1,min_alpha=0.001)
    print('Model Initialized.')

    # Building vocabulary.
    model.build_vocab(documents) 
    print('Vocabulary size: ',len(model.wv.vocab.keys()))
    
    # Training model.
    for epoch in range(1,101):
        model.train(documents,total_examples=len(documents),epochs=1)
        if epoch==1 or epoch%10==0:
            print('Epoch :',epoch, cosine_similarity([model.docvecs[94],model.docvecs[519]])[0][1])
    print('Model Trained.')

    # Saving model.
    file = open('Models/model_'+collection_name,'wb')
    model.save(file)
    file.close()
    print('Model Saved.\n\n')


In [7]:
for collection in ['dataset_agriculture', 'dataset_development', 'dataset_environment', 'dataset_industrialization', 'dataset_lifestyle']:
    DT2V_train(collection[8:])

Agriculture
Documents Collected.
Model Initialized.
Vocabulary size:  106532
Epoch : 1 0.33052683
Epoch : 10 0.54881465
Epoch : 20 0.8100436
Epoch : 30 0.9042015
Epoch : 40 0.9568333
Epoch : 50 0.9703056
Epoch : 60 0.9809509
Epoch : 70 0.9854206
Epoch : 80 0.9896449
Epoch : 90 0.99295413
Epoch : 100 0.9943576
Model Trained.
Model Saved.


Development
Documents Collected.
Model Initialized.
Vocabulary size:  41979
Epoch : 1 0.67967117
Epoch : 10 0.7198534
Epoch : 20 0.7666781
Epoch : 30 0.83860666
Epoch : 40 0.8622291
Epoch : 50 0.86992246
Epoch : 60 0.9104696
Epoch : 70 0.9031308
Epoch : 80 0.92665
Epoch : 90 0.9553381
Epoch : 100 0.9545691
Model Trained.
Model Saved.


Environment
Documents Collected.
Model Initialized.
Vocabulary size:  122382
Epoch : 1 0.30987102
Epoch : 10 0.50402427
Epoch : 20 0.7634925
Epoch : 30 0.8477136
Epoch : 40 0.89880735
Epoch : 50 0.92165256
Epoch : 60 0.9426516
Epoch : 70 0.9553761
Epoch : 80 0.96500194
Epoch : 90 0.97345155
Epoch : 100 0.97951436
Model 